<a href="https://colab.research.google.com/github/s3xmaxman/EasyDeepFake/blob/main/EASYDeepFake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title **Install/インストール**
!git clone https://github.com/s0md3v/roop.git
%cd roop
!pip install onnxruntime-gpu && pip install -r requirements.txt
!wget https://huggingface.co/MonsterMMORPG/SECourses/resolve/main/inswapper_128.onnx
%cd /content
!rm -rf CodeFormer
!git clone https://github.com/cedro3/CodeFormer.git

%cd CodeFormer
!pip install -r requirements.txt
!python basicsr/setup.py develop

!python scripts/download_pretrained_models.py facelib
!python scripts/download_pretrained_models.py CodeFormer

from function import *

! mkdir download
%cd /content/roop

In [ ]:
!python run.py -f face.png -t video.mp4 -o face_changed_video.mp4 --keep-frames --keep-fps --gpu

In [ ]:
#@title **Loding Movie/動画の読み込み**
#@markdown ■60seconds or less(in case of 30fps)/60秒以内（30fpsの場合）として下さい
!cp /content/roop/face_changed_video.mp4 /content/CodeFormer/movie
%cd /content/CodeFormer
# 設定
video = 'face_changed_video.mp4'
video_file = 'movie/'+video
image_dir='inputs/frame/'
image_file='%s.png'

# video_2_images
reset_folder('inputs/frame')
fps, i, interval = video_2_images(video_file, image_dir, image_file)

# スタートフレーム表示
from google.colab.patches import cv2_imshow
img = cv2.imread('inputs/frame/000000.png')
cv2_imshow(img)

# パラメータ表示
print('fps = ', fps)
print('frames = ', i)
print('interval = ', interval)

In [ ]:
from types import FrameType
#@title **Higher resolution/高画質化**
#@markdown　■CodeFormer Weight/ CodeFormerのウェイト

# 設定
input_folder = 'inputs/frame'
w = 0.5 #@param {type:"slider", min:0.1, max:0.9, step:0.1}
reset_folder('results/frame_'+str(w))

# フレームの高画質化
! python inference_codeformer.py --w $w\
                                  --test_path $input_folder\
                                  --bg_upsampler realesrgan\
                                  --face_upsample

# ログ・クリア
clear_output()

# 動画の作成
print('makeing movie...')
fps_r = fps/interval
file_path = 'results/frame_'+str(w)+'/final_results/%06d.png'
! ffmpeg -y -r $fps_r -i $file_path -vcodec libx264 -pix_fmt yuv420p -loglevel error out.mp4

# 音声の抽出・付加
print('preparation for sound...')
! ffmpeg -y -i $video_file -loglevel error sound.mp3
! ffmpeg -y -i out.mp4 -i sound.mp3 -loglevel error output.mp4

# 動画の再生
print('waiting for play movie...')
display_mp4('output.mp4')

In [ ]:
#@title **Download/ダウンロード**
from google.colab import files
import shutil

dst_filepath = 'download/'+os.path.splitext(video)[0]+'_s.mp4'
shutil.copy('output.mp4', dst_filepath)
files.download(dst_filepath)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>